In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This analysis used Graph theory to explore complex relationships


<p> It litereally follows every step of the tutorial 
http://programminghistorian.github.io/ph-submissions/lessons/published/exploring-and-analyzing-network-data-with-python
</p>

In [2]:
df = pd.read_csv("Z:00_ETL/CustomerBehaviour/rawdata2.txt", sep = "\t", encoding = "ISO-8859-1")

In [69]:
print(df.shape)
df.tail()

(15695999, 9)


,series_or_movie_name,encrypted_customer_id,offer_group_desc,first_genre,entity_type,content_age,really_frist_stream,transaction_date_local,display_price
15695994,Royal Pains [dt./OV],AWHGX01JJFP1A,PRIME,comedy,TV Show,NaN,2016-10-01,NaN,NaN
15695995,Cloverfield [dt./OV],AXNCSJSOYNU7Q,PRIME,action,Movie,NaN,2015-11-14,NaN,NaN
15695996,Mamma Mia! - Der Film [dt./OV],AXNCSJSOYNU7Q,PRIME,comedy,Movie,NaN,2017-12-23,NaN,NaN
15695997,Walhalla Rising,AY4FUNR962OYO,PRIME,adventure,Movie,NaN,2014-12-21,NaN,NaN
15695998,The Zero Theorem [dt./OV],AYYHG5BNPF7M5,PRIME,comedy,Movie,NaN,2016-12-08,NaN,NaN


In [70]:
df['encrypted_customer_id'].nunique()

184163

In [71]:
df['series_or_movie_name'].nunique()

36856

# Take sample to create toy sample

In [72]:
toy = df.sample(frac=0.00005)

In [73]:
toy.shape
toy.head()

,series_or_movie_name,encrypted_customer_id,offer_group_desc,first_genre,entity_type,content_age,really_frist_stream,transaction_date_local,display_price
11601855,Rocky IV - Der Kampf des Jahrhunderts [dt./OV],A3L5FUF1Q0RXHW,PRIME,drama,Movie,NaN,2017-10-03,NaN,NaN
4653264,"Dr. Seltsam - Oder wie ich lernte, die Bombe z...",A32UD777OIE60L,PRIME,comedy,Movie,NaN,2014-03-29,NaN,NaN
7712530,Pirates of the Caribbean: Salazars Rache (Teil...,A33J25LOCDUFI7,RENTAL,action,Movie,New Release,2017-10-09,2017-10-09,4.99
1355429,Cat. 8 - Wenn die Erde verglÃ¼ht [dt./OV],A3MYQ7D03AV6L5,PRIME,NaN,Movie,NaN,2016-07-13,NaN,NaN
14323016,12 Years a Slave [dt./OV],AEPNV4KBETH18,PRIME,biography,Movie,NaN,2016-11-28,NaN,NaN


In [74]:
toy['encrypted_customer_id'].nunique()

782

In [76]:
toy['encrypted_customer_id'].unique()[0:10]

array(['A3L5FUF1Q0RXHW', 'A32UD777OIE60L', 'A33J25LOCDUFI7',
       'A3MYQ7D03AV6L5', 'AEPNV4KBETH18', 'A1QAQYQ2L7GYD1',
       'A3A3CYVZ4V22DH', 'A1TWSGS2678O1I', 'A117K0WR1SAYRH',
       'A2I3QBBPXQYKP7'], dtype=object)

In [77]:
sample_df = df[df['encrypted_customer_id'].isin(toy['encrypted_customer_id'].unique())]

In [78]:
print(sample_df.shape)
sample_df.head()

(211037, 9)


,series_or_movie_name,encrypted_customer_id,offer_group_desc,first_genre,entity_type,content_age,really_frist_stream,transaction_date_local,display_price
92,Unforgettable,A2RKGDNWQLTS3L,PRIME,crime,TV Show,NaN,2015-09-26,NaN,NaN
156,KÃ¶nigin der WÃ¼ste [dt./OV],A5NXSHULCUMPO,PRIME,biography,Movie,NaN,2017-11-12,NaN,NaN
192,Ashby [dt./OV],A2CQOQSMV34PBS,PRIME,comedy,Movie,NaN,2016-06-22,NaN,NaN
229,Magic Mike [dt./OV],A12I03Q3SPPW7A,PRIME,comedy,Movie,NaN,2017-07-01,NaN,NaN
255,Alien: Covenant [dt./OV],A3BO04J9XF4KWH,RENTAL,horror,Movie,New Release,2017-09-28,2017-09-28,4.99


In [79]:
sample_df['first_genre'].unique()

array(['crime', 'biography', 'comedy', 'horror', 'action', 'drama', nan,
       'adventure', 'animation', 'mystery', 'thriller', 'romance',
       'documentary', 'western', 'fantasy', 'sci_fi', 'family', 'music',
       'adult', 'musical', 'short', 'history', 'sport', 'war',
       'reality_tv', 'film_noir', 'talk_show', 'news'], dtype=object)

In [80]:
sample_df['entity_type'].unique()

array(['TV Show', 'Movie', 'Educational', 'Live Sporting Event UX 1',
       'Short Film', nan, 'Default', 'Concert/Performance', 'Clip',
       'News and Commentary', 'Ambient', 'Other', 'Sporting Event',
       'Review', 'Music Video', 'Interview', 'Live Sporting Event UX 2'], dtype=object)

In [81]:
sample_df['offer_group_desc'].unique()

array(['PRIME', 'RENTAL', 'PURCHASE'], dtype=object)

# Construct nodes and edges

In [82]:
nodereader = sample_df[['series_or_movie_name','offer_group_desc','first_genre','entity_type','content_age']]

In [83]:
edgereader = sample_df[['series_or_movie_name','encrypted_customer_id']]
edgereader.columns = ['source','target']

In [84]:
nodereader.head()

,series_or_movie_name,offer_group_desc,first_genre,entity_type,content_age
92,Unforgettable,PRIME,crime,TV Show,NaN
156,KÃ¶nigin der WÃ¼ste [dt./OV],PRIME,biography,Movie,NaN
192,Ashby [dt./OV],PRIME,comedy,Movie,NaN
229,Magic Mike [dt./OV],PRIME,comedy,Movie,NaN
255,Alien: Covenant [dt./OV],RENTAL,horror,Movie,New Release


In [85]:
edgereader.head()

,source,target
92,Unforgettable,A2RKGDNWQLTS3L
156,KÃ¶nigin der WÃ¼ste [dt./OV],A5NXSHULCUMPO
192,Ashby [dt./OV],A2CQOQSMV34PBS
229,Magic Mike [dt./OV],A12I03Q3SPPW7A
255,Alien: Covenant [dt./OV],A3BO04J9XF4KWH


# Import networkx

In [86]:
import networkx as nx

from operator import itemgetter
import community

In [87]:
node_names = [i for i in nodereader.iloc[:,0]]

In [88]:
node_names[0:5]

['Unforgettable',
 'KÃ¶nigin der WÃ¼ste [dt./OV]',
 'Ashby [dt./OV]',
 'Magic Mike [dt./OV]',
 'Alien: Covenant [dt./OV]']

In [89]:
edges = list(zip(edgereader.source, edgereader.target)) #create tuple for every pair from the dataset

In [90]:
print(len(node_names))
print(len(edges))

211037
211037


# Basics of NetworkX: Creating the Graph

In [91]:
G = nx.Graph() #creates empty graph, initiliasize a graph object

#Add list of nodes and edges
G.add_nodes_from(node_names)
G.add_edges_from(edges)

In [92]:
print(nx.info(G)) #print basic info about the newly create graph

Name: 
Type: Graph
Number of nodes: 13332
Number of edges: 206202
Average degree:  30.9334


# Adding attributes

In [93]:
#initialize empty dictionaries

offer_group_desc_dict = {}
first_genre_dict = {}
entity_type_dict = {}

In [94]:
#Make nodes generator, i.e. convert each row of dataframe into a list
nodes = nodereader.values.tolist() 

In [95]:
nodes[0:4]

[['Unforgettable', 'PRIME', 'crime', 'TV Show', nan],
 ['KÃ¶nigin der WÃ¼ste [dt./OV]', 'PRIME', 'biography', 'Movie', nan],
 ['Ashby [dt./OV]', 'PRIME', 'comedy', 'Movie', nan],
 ['Magic Mike [dt./OV]', 'PRIME', 'comedy', 'Movie', nan]]

In [96]:
for node in nodes:
    offer_group_desc_dict[node[0]] = node[1]
    first_genre_dict[node[0]] = node[2]
    entity_type_dict[node[0]] = node[3]

<p> After having each attribute in a node, add attributes to Graph using set_node_attributes function
which takes 3 variables : the graph to which attributes are added , name of the attribute, dict of attributes </p>

In [97]:
nx.set_node_attributes(G,  offer_group_desc_dict, 'offer_group')
nx.set_node_attributes(G, first_genre_dict, 'genre')
nx.set_node_attributes(G, entity_type_dict, 'entity_type')

<p> Now all nodes have attributes, which can be accessed at any time </p>
<p> For example, print all genres of the nodes by looping through them and accessing genre attribute </p>

In [31]:
# for n in G.nodes():
#     print(n, G.node[n]['genre'])

# Metrics available in Networkx

## Density

<p> A good metric to begin with is network density. This is simply the ratio of actual edges in the network to all possible edges in the network. In an undirected network like this one, there could be a single edge between any two nodes, but as you saw in the visualization, only a few of those possible edges are actually present. Network density gives you a quick sense of how closely knit your network is.
</p>

In [98]:
density = nx.density(G)
print("Network density: ", density)

Network density:  0.0023204105723001974


<p>
In this case, the density of our network is quite low. On a scale of 0 to 1, not a very dense network, which comports with what you can see in the visualization.8 A 0 would mean that there are no connections at all, and a 1 would indicate that all possible edges are present (a perfectly connected network): this Quaker network is on the lower end of that scale, but still far from 0.
</p>


<p> A shortest path measurement is a bit more complex. It calculates the shortest possible series of nodes and edges that stand between any two nodes, something hard to see in large network visualizations. This measure is essentially finding friends-of-friends—if my mother knows someone that I don’t, then mom is the shortest path between me and that person.
</p>

## Shortest path measurement

<p> 
It calculates the shortest possible series of nodes and edges that stand between any two nodes, something hard to see in large network visualizations. This measure is essentially finding friends-of-friends—if my mother knows someone that I don’t, then mom is the shortest path between me and that person.
</p>


<p> 
To calculate a shortest path, you’ll need to pass several input variables (information you give to a Python function): the whole graph, your source node, and your target node. </p>

In [101]:
#mission_impossible_path = nx.shortest_path(G, source="Mission: Impossible - Rogue Nation [dt./OV]",
#                                       target="A2Y5MIXLAKE8B4")

#print("Shortest path between Mission Impossible and customer:", mission_impossible_path)

## Triadic closure

<p> The final structural calculation you will make on this network concerns the concept of triadic closure. Triadic closure supposes that if two people know the same person, they are likely to know each other. If C knows both A and B, then B and A may very well know each other, completing a triangle in the visualization of three edges connecting A, B, and C. The number of these enclosed triangles in the network can be used to find clusters and communities of individuals that all know each other fairly well.
</p>

<p> 
One way of measuring triadic closure is called clustering coefficient because of this clustering tendency, but the structural network measure you will learn is known as transitivity.11 Transitivity is the ratio of all triangles over all possible triangles. A possible triangle exists when one person (C) knows two people (A and B). So transitivity, like density, expresses how interconnected a graph is in terms of a ratio of actual over possible connections. Remember, measurements like transitivity and density concern likelihoods rather than certainties. All the outputs of your Python script must be interpreted, like any other object of research. Transitivity allows you a way of thinking about all the relationships in your graph that may exist but currently do not.</p>

In [102]:
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

Triadic closure: 0


## Centrality

<p> 
After getting some basic measures of the entire network structure, a good next step is to find which nodes are the most important ones in your network. In network analysis, measures of the importance of nodes are referred to as centrality measures. Because there are many ways of approaching the question “Which nodes are the most important?” there are many different ways of calculating centrality. Here you’ll learn about three of the most common centrality measures: degree, betweenness centrality, and eigenvector centrality.</p>

<p> 
Degree is the simplest and the most common way of finding important nodes. A node’s degree is the sum of its edges. If a node has three lines extending from it to other nodes, its degree is three. Five edges, its degree is five. It’s really that simple. Since each of those edges will always have a node on the other end, you might think of degree as the number of people to which a given person is directly connected. The nodes with the highest degree in a social network are the people who know the most people. These nodes are often referred to as hubs, and calculating degree is the quickest way of identifying hubs.</p>

<p> 
Calculating centrality for each node in NetworkX is not quite as simple as the network-wide metrics above, but it still involves one-line commands. All of the centrality commands you’ll learn in this section produce dictionaries in which the keys are nodes and the values are centrality measures. That means they’re ready-made to add back into your network as a node attribute, like you did in the last section. Start by calculating degree and adding it as an attribute to your network.</p>

In [103]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

<p> Since you’re already in Python, you can sort and compare them. You can use the built-in function sorted() to sort a dictionary by its keys or values and find the top twenty nodes ranked by degree. To do this you’ll need to use itemgetter, which we imported back at the beginning of the tutorial. Using sorted and itemgetter, you can sort the dictionary of degrees like this: </p>

In [104]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

<p> There’s a lot going on behind the scenes here, but just concentrate on the three input variables you gave to sorted(). The first is the dictionary, degree_dict.items(), you want to sort. The second is what to sort by: in this case, item “1” is the second item in the pair, or the value of your dictionary. Finally, you tell sorted() to go in reverse so that the highest degree nodes will be first in the resulting list. Once you’ve created this sorted list, you can loop through it, and use list slicing3 to get only the first 10 nodes:
</p>

In [105]:
sorted_degree[0:20]

[('AW2WS61CEG9KS', 2025),
 ('A3LOBPOUT4WHCJ', 1938),
 ('A29N9NB8M8IDOL', 1476),
 ('A1J9HH17090ZWJ', 1408),
 ('A3B94VKCIETZIS', 1095),
 ('A3R9AWXLIOEMEB', 1084),
 ('A3LK99GFXS8SAH', 1006),
 ('A217O7RDQY6707', 1005),
 ('A2JBS3RPT95Y5K', 999),
 ('A2MI2QF6R97APD', 980),
 ('A31R1HI2BHK6XV', 978),
 ('A3XZPPONNRE1G', 968),
 ('AH0GBMUNY3L6I', 966),
 ('A18C8DMG8LGAQK', 953),
 ('A1PWFZMF1M4RM1', 884),
 ('A2UPWZ58O0EVJZ', 882),
 ('A1B4S6RWZGSMKD', 879),
 ('A2W9OBBI7KTWD5', 863),
 ('AED7E0V4S26CP', 850),
 ('A1TSR4VWF1UHAK', 844)]

<p> Most social networks will have just a few hubs of very high degree, with the rest of similar, much lower degree. 
Degree can tell you about the biggest hubs, but it can’t tell you that much about the rest of the nodes. And in many cases, those hubs it’s telling you about are not especially surprising. </p>


<p> 
Thankfully there are other centrality measures that can tell you about more than just hubs. Eigenvector centrality is a kind of extension of degree—it looks at a combination of a node’s edges and the edges of that node’s neighbors. Eigenvector centrality cares if you are a hub, but it also cares how many hubs you are connected to. It’s calculated as a value from 0 to 1: the closer to one, the greater the centrality. Eigenvector centrality is useful for understanding which nodes can get information to many other nodes quickly. If you know a lot of well-connected people, you could spread a message very efficiently. If you’ve used Google, then you’re already somewhat familiar with Eigenvector centrality. Their PageRank algorithm uses an extension of this formula to decide which webpages get to the top of its search results.</p>


<p> 
Betweenness centrality is a bit different from the other two measures in that it doesn’t care about the number of edges any one node or set of nodes has. Betweenness centrality looks at all the shortest paths that pass through a particular node (see above). To do this, it must first calculate every possible shortest path in your network, so keep in mind that betweenness centrality will take longer to calculate than other centrality measures (but it won’t be an issue in a dataset of this size). Betweenness centrality, which is also expressed on a scale of 0 to 1, is fairly good at finding nodes that connect two otherwise disparate parts of a network. If you’re the only thing connecting two clusters, every communication between those clusters has to pass through you. In contrast to a hub, this sort of node is often referred to as a broker. Betweenness centrality is not the only way of finding brokerage (and other methods are more systematic), but it’s a quick way of giving you a sense of which nodes are important not because they have lots of connections themselves but because they stand between groups, giving the network connectivity and cohesion.</p>

In [ ]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
#eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality, failed to converge

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
#nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

In [51]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

In [52]:
sorted_betweenness[0:10]

[('ATZ1OTW8WZHIW', 0.4072766370077794),
 ('A2Y5MIXLAKE8B4', 0.3216008825263349),
 ('A34ZRFKWXBL8Y2', 0.29014995531988697),
 ('A2716J9WBUOI33', 0.21205499237393158),
 ('A18XXKP4LZ4V7J', 0.17334583354691535),
 ('A281KGD1HQ8LCO', 0.15412680400076717),
 ('A2WYMF249QFYDR', 0.11146965097261535),
 ('A30BWWHINAW0EF', 0.05941718216131657),
 ('Die Bestimmung - Allegiant [dt./OV]', 0.016925353269131683),
 ('The Walking Dead [dt./OV]', 0.01218492285140067)]

In [56]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

Name: ATZ1OTW8WZHIW | Betweenness Centrality: 0.4072766370077794 | Degree: 416
Name: A2Y5MIXLAKE8B4 | Betweenness Centrality: 0.3216008825263349 | Degree: 327
Name: A34ZRFKWXBL8Y2 | Betweenness Centrality: 0.29014995531988697 | Degree: 317
Name: A2716J9WBUOI33 | Betweenness Centrality: 0.21205499237393158 | Degree: 231
Name: A18XXKP4LZ4V7J | Betweenness Centrality: 0.17334583354691535 | Degree: 181
Name: A281KGD1HQ8LCO | Betweenness Centrality: 0.15412680400076717 | Degree: 180
Name: A2WYMF249QFYDR | Betweenness Centrality: 0.11146965097261535 | Degree: 143
Name: A30BWWHINAW0EF | Betweenness Centrality: 0.05941718216131657 | Degree: 80
Name: Die Bestimmung - Allegiant [dt./OV] | Betweenness Centrality: 0.016925353269131683 | Degree: 6
Name: The Walking Dead [dt./OV] | Betweenness Centrality: 0.01218492285140067 | Degree: 5
Name: Nerve [dt./OV] | Betweenness Centrality: 0.011556427037288325 | Degree: 5
Name: The Shannara Chronicles [dt./OV] | Betweenness Centrality: 0.011430597992887925

# Advanced networkx: Community detection with modularity

<p> 
Another common thing to ask about a network dataset is what the subgroups or communities are within the larger social structure. Is your network one big, happy family where everyone knows everyone else? Or is it a collection of smaller subgroups that are only connected by one or two intermediaries? The field of community detection in networks is designed to answer these questions. There are many ways of calculating communities, cliques, and clusters in your network, but the most popular method currently is modularity. Modularity is a measure of relative density in your network: a community (called a module or modularity class) has high density relative to other nodes within its module but low density with those outside. Modularity gives you an overall score of how fractious your network is, and that score can be used to partition the network and return the individual communities</p>

<p> 
Community detection and partitioning in NetworkX requires a little more setup than some of the other metrics. There are some built-in approaches to community detection (like minimum cut, but modularity is not included with NetworkX. Fortunately there’s an additional python module you can use with NetworkX, which you already installed and imported at the beginning of this tutorial. You can read the full documentation for all of the functions it offers, but for most community detection purposes you’ll only want best_partition()
</p>

In [ ]:
communities = community.best_partition(G)


<p> The above code will create a dictionary just like the ones created by centrality functions. best_partition() tries to determine the number of communities appropriate for the graph, and assigns each node a number (starting at 0), corresponding to the community it’s a member of. You can add these values to your network in the now-familiar way: </p>

In [58]:
nx.set_node_attributes(G, communities,'modularity')

<p> In smaller networks like this one, a common task is to find and list all of the modularity classes and their members. You can do this by manipulating the communities dictionary. You’ll need to reverse the keys and values of this dictionary so that the keys are the modularity class numbers and the values are lists of names. You can do so like this: </p>

In [66]:
modularity = {} #creates empty dictionary
for k, v in communities.items(): #loop thru the community dict
    if v not in modularity:
        modularity[v] = [k] #add new key for a modularity class the code has not seen before
    else:
        modularity[v].append(k) #append a name to the list for a modularity class the code has already seen
        
for k,v in modularity.items():
    if len(v) > 2: #filter out modularity classes with 2 or fewer nodes
        print("Class" + str(k) + ":", v[0:2]) #print the classes and their members

Class0: ['Pandemic [dt./OV]', 'Love & Teleportation [OV]']
Class1: ['Ich Einfach unverbesserlich 2:  3 Mini-Movies Collection [dt./OV]', 'Die HÃ¼terin der Wahrheit â\x80\x94 Dinas Bestimmung']
Class2: ['Bergfest', 'The Neighbors']
Class3: ['Mission: Impossible - Rogue Nation [dt./OV]', 'Taboo [dt./OV]']
Class4: ['Zambezia -  In jedem steckt ein kleiner Held [dt./OV]', 'Housebound [dt./OV]']
Class5: ['Chaos [dt./OV]', 'Scooby-Doo: Mystery Incorporated']
Class6: ['Full Metal Jacket [dt./OV]', '42 [dt./OV]']
Class7: ['Shame [dt./OV]', 'Minions [dt./OV]']


<p> Notice in the code above that you are filtering out any modularity classes with two or fewer nodes, in the line if len(v) > 2. You’ll remember from the visualization that there were lots of small components of the network with only two nodes. Modularity will find these components and treat them as separate classes (since they’re not connected to anything else). By filtering them out, you get a better sense of the larger modularity classes within the network’s main component. </p>

# Export graph

In [65]:
nx.write_gexf(G, 'sample_movies.gexf')